In [5]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

matplotlib.style.use('ggplot') 
#pd.set_option('display.max_columns', None)


In [6]:
state_abbrev = {
    'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA',
    'Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','Florida': 'FL','Georgia': 'GA',
    'Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS',
    'Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA',
    'Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO','Montana': 'MT',
    'Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM',
    'New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK',
    'Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC',
    'South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT',
    'Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY',
    'District of Columbia': 'DC'
}

In [7]:
# Comparison Data from David Leip's Elections Atlas
# Source: http://uselectionatlas.org/RESULTS/

# 2004
# George W. Bush	Richard Cheney	Republican	62,039,572	50.73%	286	53.2%  
# John Kerry	John Edwards	Democratic	59,027,115	48.26%	251	46.7%  
# Total	122,303,590		538	

# 2008
# Barack H. Obama	Joseph R. Biden, Jr.	Democratic	69,499,428	52.86%	365	67.8%  
# John S. McCain, III	Sarah H. Palin	Republican	59,950,323	45.60%	173	32.2%  
# Total	131,473,705		538	

# 2012
# Barack H. Obama	Joseph R. Biden, Jr.	Democratic	65,918,507	51.01%	332	61.7%  
# Willard Mitt Romney	Paul Ryan	Republican	60,934,407	47.15%	206	38.3%  
# Total	129,237,642		538	

# 2016
# Donald J. Trump	Michael R. Pence	Republican	62,985,106	45.94%	304	56.5%  
# Hillary Clinton	Tim Kaine	Democratic	65,853,625	48.03%	227	42.2%  
# Total	137,100,229		538	


In [8]:
# Load the 2012, 2016 Voting and Turnout data
# Source: https://github.com/kyaroch/2012_and_2016_presidential_election_results_by_county

voting1_df = pd.read_csv('./data/2012_and_2016_presidential_election_results_by_county.csv',
                         converters={'fips_code': lambda x: str(x)})

columns = ['state_postal_abbrev', 'fips_code', 'county_name', '2012_total_votes', 
           'obama_votes', 'romney_votes', '2016_total_votes', 'clinton_votes', 
           'trump_votes', '2012_adult_pop', '2016_extrapolated_adult_pop', 
           '2012_turnout', '2016_turnout']

renames = ['state',  'fips_code', 'county_name',  'total_2012', 'dem_2012', 
           'rep_2012','total_2016', 'dem_2016', 'rep_2016', 'vap_2012','vap_2016',
           'turnout_2012','turnout_2016']

voting1_df = voting1_df[columns]
voting1_df.rename(columns = dict(zip(columns, renames)), inplace=True)

#Fix fips_code, add leading zero
voting1_df['fips_code'] = voting1_df['fips_code'].apply(lambda x: '0' + x if len(x) < 5 else x)

voting1_df['county_name'] = voting1_df['county_name'] + " County"

# voting1_df[voting1_df['2016_results_official'] == 0].count()
# 385 unofficial results

#Close enough to above numbers from Leip's election atlas. 
print(voting1_df.sum(numeric_only=True).apply(lambda x: '%.2f' % x))

#Turn year into column
year_dfs = []

for year in [2012, 2016]:
    cols = 'state fips_code county_name total_{0} dem_{0} rep_{0} vap_{0} turnout_{0}'.format(year)
    cols = cols.split(' ')
    temp_df = voting1_df[cols].copy()
    temp_df['year'] = year
    renames = dict(zip(cols, ['state', 'fips_code', 'county_name',
                              'county_num', 'dem_num', 'rep_num', 'vap', 'turnout']))
    temp_df.rename(columns=renames, inplace=True)
    year_dfs.append(temp_df)

voting1_df = pd.concat(year_dfs)

voting1_df['dem_margin'] = (voting1_df['dem_num'] - voting1_df['rep_num'])/voting1_df['county_num']

#voting1_df.sort_values(by='dem_margin')
voting1_df.head()


total_2012      128957555.00
dem_2012         65791440.00
rep_2012         60767069.00
total_2016      136352670.00
dem_2016         65689549.00
rep_2016         62778232.00
vap_2012        239849637.00
vap_2016        249735736.00
turnout_2012         1751.15
turnout_2016         1775.76
dtype: object


,state,fips_code,county_name,county_num,dem_num,rep_num,vap,turnout,year,dem_margin
0,AL,01001,Autauga County,23973,6363,17379,40874,0.5865,2012,-0.459517
1,AL,01003,Baldwin County,85491,18424,66016,147416,0.5799,2012,-0.556690
2,AL,01005,Barbour County,11517,5912,5550,21334,0.5398,2012,0.031432
3,AL,01007,Bibb County,8420,2202,6132,17796,0.4731,2012,-0.466746
4,AL,01009,Blount County,24060,2970,20757,43876,0.5484,2012,-0.739277


In [9]:
# Load Voting Age Population 2005 to 2009 average, can't find yearly VAP data
# Source: https://www.census.gov/rdo/data/voting_age_population_by_citizenship_and_race_cvap.html

# 'latin-1' deals with e-acute: https://stackoverflow.com/questions/5552555
population2009_df = pd.read_csv('./data/CVAP_CSV_Format_2005-2009/County.csv', encoding='latin-1')

population2009_df['fips_code'] = population2009_df['GEOID'].apply(lambda x: x[7:])
population2009_df = population2009_df[(population2009_df['LNTITLE'] == 'Total')]
population2009_df.rename(columns={'CVAP_EST':'vap'}, inplace=True)
population2009_df = population2009_df[['fips_code', 'vap']]  

population2009_df.head()

,fips_code,vap
0,01001,35315
13,01003,128945
26,01005,22285
39,01007,16255
52,01009,41095


In [10]:
# Load 2004-2012 Voting data 
# Source: https://github.com/helloworlddata/us-presidential-election-county-results
columns = ['year', 'state', 'county', 'fips', 'vote_rep', 'vote_dem', 'vote_total']
renames = ['year', 'state', 'county_name', 'fips_code', 'rep_num', 'dem_num', 'county_num']


voting2_df = pd.read_csv('./data/us-presidential-election-county-results-2004-through-2012.csv')

voting2_df = voting2_df[columns]
voting2_df.rename(columns=dict(zip(columns, renames)), inplace=True)

# Fix Laclede County Results, otherwise state switches to D
# https://en.wikipedia.org/wiki/United_States_presidential_election_in_Missouri,_2008
# Laclede	32.4%	5,218	67.6%	10,875	16,093
voting2_df.loc[(voting2_df['state'] == 'MO') & 
           (voting2_df['year'] == 2008) & 
           (voting2_df['county_name'] == 'Laclede County'), 'county_num'] = 16093

voting2_df['dem_margin'] = (voting2_df['dem_num'] - voting2_df['rep_num'])/voting2_df['county_num'] 

voting2_df = voting2_df[(voting2_df['year'].isin([2004, 2008])) &
                        (voting2_df['state'] != 'AK')]

voting2_df = pd.merge(voting2_df, population2009_df, on='fips_code')

voting2_df['turnout'] = voting2_df['county_num'] / voting2_df['vap']

#Close enough to above numbers from Leip's election atlas. 
#print(voting2_df[.sum(numeric_only=True).apply(lambda x: '%.2f' % x))


voting2_df.head()

,year,state,county_name,fips_code,rep_num,dem_num,county_num,dem_margin,vap,turnout
0,2004,AL,Autauga County,01001,15196,4758,20081,-0.519795,35315,0.568625
1,2008,AL,Autauga County,01001,17403,6093,23641,-0.478406,35315,0.669432
2,2004,AL,Baldwin County,01003,52971,15599,69320,-0.539123,128945,0.537594
3,2008,AL,Baldwin County,01003,61271,19386,81413,-0.514476,128945,0.631378
4,2004,AL,Barbour County,01005,5899,4832,10777,-0.099007,22285,0.483599


In [11]:
# Data quality checks:

# voting2_df.sort_values(by='turnout')  #by='dem_margin'

# voting2_df[(voting2_df['turnout'] > 1) & (voting2_df['year'] == 2004)]

# year	state	county_name	fips_code	rep_num	dem_num	county_num	dem_margin	vap	turnout
# 484	2004	CO	Hinsdale County	08053	355	236	602	-0.197674	490	1.228571
# 2254	2004	LA	Saint Bernard Parish	22087	19597	9956	29838	-0.323111	27540	1.083442
# 3254	2004	NE	Arthur County	31005	240	24	266	-0.812030	235	1.131915
# 3554	2004	NM	Harding County	35021	380	259	644	-0.187888	505	1.275248
# 4294	2004	OK	McIntosh County	40091	9946	6933	16879	-0.178506	15420	1.094617
# 4296	2004	OK	Major County	40093	10041	3742	13783	-0.457012	5405	2.550046
# 4298	2004	OK	Marshall County	40095	7472	3684	11156	-0.339548	10340	1.078917

# voting2_df[(voting2_df['turnout'] > 1) & (voting2_df['year'] == 2008)]
# Possible this will affect RI results, 168,000 will be filtered out of a population of 1 million
# We'll see how it works.

# 	year	state	county_name	fips_code	rep_num	dem_num	county_num	dem_margin	vap	turnout
# 485	2008	CO	Hinsdale County	08053	344	240	599	-0.173623	490	1.222449
# 1427	2008	IN	LaPorte County	18091	17918	28258	208757	0.049531	82565	2.528396
# 3193	2008	MT	McCone County	30055	2822	1607	4567	-0.266039	1330	3.433835
# 3255	2008	NE	Arthur County	31005	217	39	263	-0.676806	235	1.119149
# 3555	2008	NM	Harding County	35021	358	260	626	-0.156550	505	1.239604
# 4573	2008	RI	Washington County	44009	49810	116156	168633	0.393434	98525	1.711576
# 4639	2008	SC	McCormick County	45065	5416	9608	15174	0.276262	8435	1.798933
# 4907	2008	TN	McNairy County	47109	23290	20209	43846	-0.070269	19350	2.265943
# 4913	2008	TN	Marion County	47115	20288	13058	33767	-0.214114	21455	1.573852

# print(
# voting2_df[voting2_df['turnout'] > 1]['county_num'].count(),
# voting2_df[voting2_df['turnout'] > 1]['county_num'].sum(),
# voting2_df[(voting2_df['turnout'] > 1) & 
#             (voting2_df['year'] == 2004)]['county_num'].sum(),
# voting2_df[(voting2_df['turnout'] > 1) & 
#             (voting2_df['year'] == 2008)]['county_num'].sum()
# )
# So there are 16 counties with turnout over 1
# and they make up 549400 votes. 
# with 476232 coming from 2008, and 73168 from 2004
# 16 549400 73168 476232

# These counties have negative turnout. . . 
# 1039	2008	HI	Kalawao County	15005	-9999	-9999	-9999	-0.000000	80	-124.987500
# 1038	2004	HI	Kalawao County	15005	-9999	-9999	-9999	-0.000000	80	-124.987500


# MO switches sides in results:
# voting2_df[(voting2_df['state'] == 'MO') & 
#            (voting2_df['year'] == 2008) &
#            ((voting2_df['turnout'] > 1) | (voting2_df['dem_margin'] > 1) | (voting2_df['dem_margin'] < -1)) ]

# This is the cause:
# 	year	state	county_name	fips_code	rep_num	dem_num	county_num	dem_margin	vap	turnout
# 3013	2008	MO	Laclede County	29105	10875	5218	2024	-2.79496	25950	0.077996
# Fixed above.  


# voting2_df['fips_code'].drop_duplicates().count()
# 3154 counties, other is 3112, so probably alaska is main difference? but that's only ~ 10-20
# Note, without AK = 3114, close enough.  2 missing ones are from other dataset that I removed.  I could look
# those counties up, then these would match exactly.  pretty good.  

# voting2_df.sort_values(by='dem_margin')

# -200% margin here.
# 4660	2008	MO	Laclede County	29105	10875	5218	2024	-2.794960
# vs https://en.wikipedia.org/wiki/United_States_presidential_election_in_Missouri,_2008
# Laclede	32.4%	5,218	67.6%	10,875	16,093
# Issue is that county num is 2023 when it should be around 16093

# https://en.wikipedia.org/wiki/Oglala_Lakota_County,_South_Dakota
# Oglala Lakota County, known as Shannon County until May 2015, is a county located 
# in the U.S. state of South Dakota. The population was 13,586 at the 2010 census. 
# Oglala Lakota County does not have its own county seat. Wikipedia
# have the same fips code though, so should be fine.  

# 2004
# print('2004 Results:')
# print('Dem: ', voting2_df[(voting2_df['year'] == 2004)]['dem_num'].sum())
# print('Rep: ', voting2_df[(voting2_df['year'] == 2004)]['rep_num'].sum())
# print('Total: ', voting2_df[(voting2_df['year'] == 2004)]['county_num'].sum())

# # 2008
# print('2008 Results:')
# print('Dem: ', voting2_df[(voting2_df['year'] == 2008)]['dem_num'].sum())
# print('Rep: ', voting2_df[(voting2_df['year'] == 2008)]['rep_num'].sum())
# print('Total: ', voting2_df[(voting2_df['year'] == 2008)]['county_num'].sum())

# 2004 Results:
# Dem:  59093576
# Rep:  62131489
# Total:  122459692
# 2008 Results:
# Dem:  69547395
# Rep:  59959177
# Total:  131502027

# Leip atlas:
# 2004
# http://uselectionatlas.org/RESULTS/
# 	George W. Bush	Richard Cheney	Republican	62,039,572	50.73%	286	53.2%
# 	John Kerry	John Edwards	Democratic	59,027,115	48.26%	251	46.7%
# Total	122,303,590		538	

# 2008
# 	Barack H. Obama	Joseph R. Biden, Jr.	Democratic	69,499,428	52.86%	365	67.8%
# 	John S. McCain, III	Sarah H. Palin	Republican	59,950,323	45.60%	173	32.2%
# Total	131,473,705		538	

# Ok, so pretty close/good enough.  

In [12]:
# Source: https://en.wikipedia.org/wiki/Electoral_College_(United_States)#Chronological_table
electoral_df = pd.read_csv('./data/state_electoral_votes.csv') #sep="\t"

# Electoral points in 2004 = 2008, 2012=2016=2020
electoral_df['votes_2016'] = electoral_df['votes_2012']
electoral_df['votes_2004'] = electoral_df['votes_2008']
electoral_df['state'] = electoral_df['state'].apply(lambda x: state_abbrev[x])

years = [2004, 2008, 2012, 2016]
electoral_dfs = []

#Stack by year
for year in years:
    cols = 'state votes_{0}'.format(year)
    cols = cols.split(' ')
    temp_df = electoral_df[cols].copy()
    temp_df['year'] = year
    temp_df.rename(columns={'votes_{}'.format(year): 'state_electoral_votes'}, inplace=True)
    electoral_dfs.append(temp_df)

electoral_df = pd.concat(electoral_dfs)

electoral_df.head()

,state,state_electoral_votes,year
0,AL,9,2004
1,AK,3,2004
2,AZ,10,2004
3,AR,6,2004
4,CA,55,2004


In [13]:
# Manually input Alaska statewide data because they don't use counties
# http://uselectionatlas.org/RESULTS/compare.php?year=2016&fips=2&f=0&off=0&elect=0&type=state

columns = ['fips_code', 'county_num', 'dem_num', 'rep_num', 
            'year', 'state', 'county_name', 'vap']

ak = [['02', '02', '02', '02'], [312598, 326197, 300495, 318608], [111025, 123594, 122640, 116454], 
      [190889, 193841, 164676, 163387], [2004, 2008, 2012, 2016], ['AK', 'AK', 'AK', 'AK'],
      ['Alaska', 'Alaska', 'Alaska', 'Alaska'], [550189, 550189, 550189, 550189]]

alaska_df = pd.DataFrame(dict(zip(columns, ak)))

alaska_df['dem_margin'] =  (alaska_df['dem_num'] - alaska_df['rep_num'])/ alaska_df['county_num']
alaska_df['turnout'] = alaska_df['county_num']/ alaska_df['vap']

alaska_df


,county_name,county_num,dem_num,fips_code,rep_num,state,vap,year,dem_margin,turnout
0,Alaska,312598,111025,02,190889,AK,550189,2004,-0.255485,0.568165
1,Alaska,326197,123594,02,193841,AK,550189,2008,-0.215351,0.592882
2,Alaska,300495,122640,02,164676,AK,550189,2012,-0.139889,0.546167
3,Alaska,318608,116454,02,163387,AK,550189,2016,-0.147306,0.579088


In [14]:
# Combine all dataframes together
turnout_df = pd.concat([voting1_df, voting2_df, alaska_df])

#Add in electoral college points for each year
turnout_df = pd.merge(turnout_df, electoral_df, on=('state', 'year'))

turnout_df.head()

,county_name,county_num,dem_margin,dem_num,fips_code,rep_num,state,turnout,vap,year,state_electoral_votes
0,Autauga County,23973,-0.459517,6363,01001,17379,AL,0.5865,40874,2012,9
1,Baldwin County,85491,-0.556690,18424,01003,66016,AL,0.5799,147416,2012,9
2,Barbour County,11517,0.031432,5912,01005,5550,AL,0.5398,21334,2012,9
3,Bibb County,8420,-0.466746,2202,01007,6132,AL,0.4731,17796,2012,9
4,Blount County,24060,-0.739277,2970,01009,20757,AL,0.5484,43876,2012,9


In [16]:
#Data output script for interactive visualization, filters out abnormal values  
out_df = turnout_df[(turnout_df['dem_margin'] <= 1.0) &
                    (turnout_df['dem_margin'] >= -1.0) &
                    (turnout_df['turnout'] >= 0.0) &
                    (turnout_df['turnout'] <= 1.0)]

# out_df = out_df[['county', 'state', 'county_num', 'turnout', 'num_rep', 'num_dem', 'num_state_dem',
#                  'num_state_rep','num_state', 'year', 'vap', 'fips_code', 'state_electoral_votes']]

out_df = out_df.round(decimals=4)

out_df.to_csv('./US_County_Level_Presidential_Results_04-16.csv', index=False)